## Importing Libraries

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

## Reading the Data

In [2]:
Project_Dir = r"D:\FLIGHT_PRICE_PREDICTION"
Data_Dir = "DATA"

In [3]:
#Fucntion to get data
def get_data(name):
    filename = f"{name}.csv"
    filepath = os.path.join(Project_Dir,Data_Dir,filename)
    return pd.read_csv(filepath)
flight = get_data("flight_price")
flight.sample(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9145,IndiGo,18/05/2019,Kolkata,Banglore,CCU → BLR,20:20,22:55,2h 35m,non-stop,No info,4804
3143,IndiGo,6/06/2019,Banglore,Delhi,BLR → DEL,18:25,21:20,2h 55m,non-stop,No info,4823
8048,IndiGo,27/05/2019,Kolkata,Banglore,CCU → BLR,09:35,12:20,2h 45m,non-stop,No info,4804
3275,IndiGo,12/06/2019,Delhi,Cochin,DEL → MAA → COK,02:00,07:25,5h 25m,1 stop,No info,5636
1600,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,17:00,01:30 22 May,8h 30m,1 stop,No info,7888


In [4]:
#Viewing information of the dataset
flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


## 3. Preliminary Analysis

### 3.1 Checking Datatypes

In [5]:
flight.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

### 3.2 Checking Duplicates

In [6]:
flight.duplicated().sum()

np.int64(220)

### 3.3 Observations

- Data type of "Date_of_Journey", "Dep_Time", "Arrival_Time" is not coreect. It should be datetime
- "Duration" and "Total_Stops" should be of numeric datatype
- There are 220 duplicate values which should be removed



## 4. Detailed Analysis

### 4.1 Getting categorical features

In [7]:
flight.select_dtypes(include = "O").columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info'],
      dtype='object')

### 4.2 Airline

In [8]:
flight.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [9]:
# Correcting the values
(
    flight
    .Airline
    .replace("Vistara Premium economy","Vistara")
    .replace("Multiple carriers Premium economy","Multiple carriers")
    .replace("Jet Airways Business","Jet Airways")
    .str.title()
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

### 4.3 Date of Journey

In [10]:
flight.Date_of_Journey.unique()

array(['24/03/2019', '1/05/2019', '9/06/2019', '12/05/2019', '01/03/2019',
       '24/06/2019', '12/03/2019', '27/05/2019', '1/06/2019',
       '18/04/2019', '9/05/2019', '24/04/2019', '3/03/2019', '15/04/2019',
       '12/06/2019', '6/03/2019', '21/03/2019', '3/04/2019', '6/05/2019',
       '15/05/2019', '18/06/2019', '15/06/2019', '6/04/2019',
       '18/05/2019', '27/06/2019', '21/05/2019', '06/03/2019',
       '3/06/2019', '15/03/2019', '3/05/2019', '9/03/2019', '6/06/2019',
       '24/05/2019', '09/03/2019', '1/04/2019', '21/04/2019',
       '21/06/2019', '27/03/2019', '18/03/2019', '12/04/2019',
       '9/04/2019', '1/03/2019', '03/03/2019', '27/04/2019'], dtype=object)

In [11]:
# Correcting the datatype
pd.to_datetime(flight.Date_of_Journey,dayfirst = True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

### 4.4 Source

In [12]:
flight.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### 4.5 Destination

In [13]:
flight.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### 4.6 Dep_Time

In [14]:
flight.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [15]:
pd.to_datetime(flight.Dep_Time,dayfirst=True).dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

In [16]:
# Viewing if there are any other value
flight.Dep_Time.str.contains("[^0-9:]").value_counts()

Dep_Time
False    10683
Name: count, dtype: int64

### 4.7 Arrival_Time

In [17]:
flight.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [18]:
# Extracting only time
pd.to_datetime(flight.Arrival_Time,dayfirst=True).dt.time

0        01:10:00
1        13:15:00
2        04:25:00
3        23:30:00
4        21:35:00
           ...   
10678    22:25:00
10679    23:20:00
10680    11:20:00
10681    14:10:00
10682    19:15:00
Name: Arrival_Time, Length: 10683, dtype: object

### 4.8 Duration

In [19]:
flight.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [20]:
# Viewing values which does not contain "m"
(
    flight
    .Duration
    .loc[lambda ser : ~ser.str.contains("m")]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [21]:
# Viewing values which does not contain "h"
(
    flight
    .Duration
    .loc[lambda ser : ~ser.str.contains("h")]
    .unique()
)

array(['5m'], dtype=object)

In [22]:
flight[flight.Duration == '5m']

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


- This is a wrong entry as duration of a flight travel can not be 5 minutes and we should delete it 

In [23]:
# Converting the values of 'Duration" column into minutes
(
    flight
    .Duration
    .drop(index = 6474)
    .str.split(" ",expand=True)
    .set_axis(['Hour','Minute'],axis=1)
    .assign(
        Hour = lambda df_ : (
            df_
            .Hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
        Minute = lambda df_ : (
             df_
            .Minute
            .str.replace("m","")
            .fillna(0)
            .astype(int)
        )
    )
    .sum(axis=1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [24]:
# Function to carry out the previous task easily during cleaning operation
def transform_duration(series):
    return(
    series
        .str.split(" ",expand=True)
        .set_axis(['Hour','Minute'],axis=1)
        .assign(
            Hour = lambda df_ : (
                df_
                .Hour
                .str.replace("h","")
                .astype(int)
                .mul(60)
            ),
            Minute = lambda df_ : (
                 df_
                .Minute
                .str.replace("m","")
                .fillna(0)
                .astype(int)
            )
        )
        .sum(axis=1)
    )

### 4.9 Total_Stops

In [25]:
flight.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [26]:
# Cleaning of the "Total Stops" column
(
    flight
    .Total_Stops
    .replace('non-stop',0)
    .str.replace(' stops','')
    .str.replace(' stop','')
    .fillna(0)
)

0        0
1        2
2        2
3        1
4        1
        ..
10678    0
10679    0
10680    0
10681    0
10682    2
Name: Total_Stops, Length: 10683, dtype: object

### 4.10 Additional_Info

In [27]:
flight.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [28]:
# Cleaning of the "Additional_Info" column
(
    flight
    .Additional_Info
    .replace('No info','No Info')
    .unique()
)

array(['No Info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

## 5. Cleaning Operations

In [29]:
# Function to carry out the whole cleaning task
def clean(df):
    return(
        df
        .drop(index = 6474)
        .drop_duplicates()
        .assign(**{
            col : df[col].str.strip() for col in df.select_dtypes(include = "O").columns})
        .rename(columns = str.lower)
        .assign(
            airline = lambda df1 : (
                df1.airline
                .replace("Vistara Premium economy","Vistara")
                .replace("Multiple carriers Premium economy","Multiple carriers")
                .replace("Jet Airways Business","Jet Airways")
                .str.title()
            ),
            date_of_journey = lambda df1 : pd.to_datetime(df1.date_of_journey,dayfirst = True),
            dep_time = lambda df1 : pd.to_datetime(df1.dep_time,dayfirst=True).dt.time,
            arrival_time = lambda df1 : pd.to_datetime(df1.arrival_time,dayfirst=True).dt.time,
            duration = lambda df1 : df1.duration.pipe(transform_duration),
            total_stops = lambda df1 : (
                df1
                .total_stops
                .replace('non-stop',0)
                .str.replace(' stops','')
                .str.replace(' stop','')
                .fillna(0)
            ),
            additional_info = lambda df1 : df1.additional_info.replace('No info','No Info')
        )
        .drop(columns = 'route')
    )

In [30]:
clean_flight = clean(flight)
clean_flight

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0,No Info,12648


## 6. Split the Data

- We will split the data into train and test data.
- Train data will be for training and hyperparameter tuning and Test data will be for testing

In [31]:
X = clean_flight.drop(['price'],axis=1)
y = clean_flight['price']

In [32]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [33]:
print(X_train.shape)
print(X_test.shape)

(7323, 9)
(3139, 9)


## 7. Exporting the Data

In [34]:
# Function for exporting
def export(x,y,name):
    filename = f"{name}.csv"
    filepath = os.path.join(r"D:\FLIGHT_PRICE_PREDICTION\DATA",filename)
    x.join(y).to_csv(filepath,index=False)
    return pd.read_csv(filepath).head()

In [35]:
export(X_train,y_train,"train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-04-01,Kolkata,Banglore,16:45:00,23:15:00,1830,3,No Info,8607
1,Multiple Carriers,2019-06-01,Delhi,Cochin,13:00:00,21:00:00,480,1,No Info,13587
2,Jet Airways,2019-05-21,Kolkata,Banglore,14:05:00,23:35:00,570,1,In-flight meal not included,10844
3,Jet Airways,2019-03-09,Delhi,Cochin,05:25:00,04:25:00,1380,2,No Info,16914
4,Jet Airways,2019-05-18,Kolkata,Banglore,16:30:00,08:15:00,945,1,In-flight meal not included,8586


In [36]:
export(X_test,y_test,"test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-06,Banglore,New Delhi,08:00:00,08:15:00,1455,1,No Info,17996
1,Spicejet,2019-06-06,Kolkata,Banglore,22:20:00,00:40:00,140,0,No Info,3873
2,Indigo,2019-03-18,Kolkata,Banglore,05:30:00,08:20:00,170,0,No Info,4462
3,Indigo,2019-06-27,Chennai,Kolkata,19:35:00,21:55:00,140,0,No Info,3597
4,Indigo,2019-05-06,Kolkata,Banglore,15:15:00,17:45:00,150,0,No Info,4804
